# Multi-GPU Mirrored Strategy

<a target="_blank" href="https://colab.research.google.com/github/LuisAngelMendozaVelasco/TensorFlow-Advanced_Techniques_Specialization/blob/master/Custom_and_Distributed_Training_with_TensorFlow/Week4/Labs/C2_W4_Lab_2_multi-GPU-mirrored-strategy.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a>

In this ungraded lab, you'll go through how to set up a Multi-GPU Mirrored Strategy. The lab environment only has a CPU but we placed the code here in case you want to try this out for yourself in a multiGPU device.

**Notes:** 
- If you are running this on Coursera, you'll see it gives a warning about no presence of GPU devices. 
- If you are running this in Colab, make sure you have selected your `runtime` to be `GPU`. 
- In both these cases, you'll see there's only 1 device that is available.  
- One device is sufficient for helping you understand these distribution strategies.

## Imports

In [1]:
import tensorflow as tf
import numpy as np
import os
from keras import datasets, Sequential, layers, losses, metrics, optimizers

## Setup Distribution Strategy

In [2]:
# Note that it generally has a minimum of 8 cores, but if your GPU has
# less, you need to set this. In this case one of my GPUs has 4 cores
os.environ["TF_MIN_GPU_MULTIPROCESSOR_COUNT"] = "4"

# If the list of devices is not specified in the
# `tf.distribute.MirroredStrategy` constructor, it will be auto-detected.
# If you have *different* GPUs in your system, you probably have to set up cross_device_ops like this
strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
print ('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of devices: 1


2024-08-24 13:09:06.983321: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1793 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


## Prepare the Data

In [3]:
# Get the data
fashion_mnist = datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Adding a dimension to the array -> new shape == (28, 28, 1)
# We are doing this because the first layer in our model is a convolutional
# layer and it requires a 4D input (batch_size, height, width, channels).
# batch_size dimension will be added later on.
train_images = train_images[..., None]
test_images = test_images[..., None]

# Normalize the images to [0, 1] range.
train_images = train_images / np.float32(255)
test_images = test_images / np.float32(255)

# Batch the input data
BUFFER_SIZE = len(train_images)
BATCH_SIZE_PER_REPLICA = 64
GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

# Create Datasets from the batches
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(BUFFER_SIZE).batch(GLOBAL_BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(GLOBAL_BATCH_SIZE)

# Create Distributed Datasets from the datasets
train_dist_dataset = strategy.experimental_distribute_dataset(train_dataset)
test_dist_dataset = strategy.experimental_distribute_dataset(test_dataset)

## Define the Model

In [4]:
# Create the model architecture
def create_model():
    model = Sequential([layers.Conv2D(32, 3, activation='relu'),
                        layers.MaxPooling2D(),
                        layers.Conv2D(64, 3, activation='relu'),
                        layers.MaxPooling2D(),
                        layers.Flatten(),
                        layers.Dense(64, activation='relu'),
                        layers.Dense(10)])
    
    return model

## Configure custom training

Instead of `model.compile()`, we're going to do custom training, so let's do that within a strategy scope.

In [7]:
with strategy.scope():
    # We will use sparse categorical crossentropy as always. But, instead of having the loss function
    # manage the map reduce across GPUs for us, we'll do it ourselves with a simple algorithm.
    # Remember -- the map reduce is how the losses get aggregated
    # Set reduction to `none` so we can do the reduction afterwards and divide byglobal batch size.
    loss_object = losses.SparseCategoricalCrossentropy(from_logits=True, reduction=None)

    def compute_loss(labels, predictions):
        # Compute Loss uses the loss object to compute the loss
        # Notice that per_example_loss will have an entry per GPU
        # so in this case there'll be 2 -- i.e. the loss for each replica
        per_example_loss = loss_object(labels, predictions)
        # You can print it to see it -- you'll get output like this:
        # Tensor("sparse_categorical_crossentropy/weighted_loss/Mul:0", shape=(48,), dtype=float32, device=/job:localhost/replica:0/task:0/device:GPU:0)
        # Tensor("replica_1/sparse_categorical_crossentropy/weighted_loss/Mul:0", shape=(48,), dtype=float32, device=/job:localhost/replica:0/task:0/device:GPU:1)
        # Note in particular that replica_0 isn't named in the weighted_loss -- the first is unnamed, the second is replica_1 etc
        print(per_example_loss)
        
        return tf.nn.compute_average_loss(per_example_loss, global_batch_size=GLOBAL_BATCH_SIZE)

    # We'll just reduce by getting the average of the losses
    test_loss = metrics.Mean(name='test_loss')

    # Accuracy on train and test will be SparseCategoricalAccuracy
    train_accuracy = metrics.SparseCategoricalAccuracy(name='train_accuracy')
    test_accuracy = metrics.SparseCategoricalAccuracy(name='test_accuracy')

    # Optimizer will be Adam
    optimizer = optimizers.Adam()

    # Create the model within the scope
    model = create_model()

## Train and Test Steps Functions

Let's define a few utilities to facilitate the training.

In [8]:
# `run` replicates the provided computation and runs it
# with the distributed input.
@tf.function
def distributed_train_step(dataset_inputs):
    per_replica_losses = strategy.run(train_step, args=(dataset_inputs,))
    #tf.print(per_replica_losses.values)

    return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None)

def train_step(inputs):
    images, labels = inputs

    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = compute_loss(labels, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_accuracy.update_state(labels, predictions)

    return loss

#######################
# Test Steps Functions
#######################

@tf.function
def distributed_test_step(dataset_inputs):
    return strategy.run(test_step, args=(dataset_inputs,))

def test_step(inputs):
    images, labels = inputs

    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss.update_state(t_loss)
    test_accuracy.update_state(labels, predictions)

## Training Loop

We can now start training the model.

In [10]:
EPOCHS = 10

for epoch in range(EPOCHS):
    # Do Training
    total_loss = 0.0
    num_batches = 0
    
    for batch in train_dist_dataset:
        total_loss += distributed_train_step(batch)
        num_batches += 1
        
    train_loss = total_loss / num_batches

    # Do Testing
    for batch in test_dist_dataset:
        distributed_test_step(batch)

    template = ("Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, " "Test Accuracy: {}")

    print (template.format(epoch+1, train_loss, train_accuracy.result()*100, test_loss.result(), test_accuracy.result()*100))

    test_loss.reset_state()
    train_accuracy.reset_state()
    test_accuracy.reset_state()

2024-08-24 13:14:57.984688: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


Epoch 1, Loss: 0.34205707907676697, Accuracy: 84.56166076660156, Test Loss: 0.3713238537311554, Test Accuracy: 86.73999786376953


2024-08-24 13:15:03.748247: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]


Epoch 2, Loss: 0.2934078276157379, Accuracy: 89.38666534423828, Test Loss: 0.3067529499530792, Test Accuracy: 88.84000396728516
Epoch 3, Loss: 0.26322174072265625, Accuracy: 90.37667083740234, Test Loss: 0.2813233733177185, Test Accuracy: 90.13999938964844
Epoch 4, Loss: 0.24071970582008362, Accuracy: 91.13333129882812, Test Loss: 0.27821969985961914, Test Accuracy: 90.11000061035156


2024-08-24 13:15:18.963832: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


Epoch 5, Loss: 0.21920861303806305, Accuracy: 91.86833190917969, Test Loss: 0.2702253758907318, Test Accuracy: 90.25
Epoch 6, Loss: 0.20070359110832214, Accuracy: 92.51166534423828, Test Loss: 0.2610572278499603, Test Accuracy: 90.79000091552734
Epoch 7, Loss: 0.18377439677715302, Accuracy: 93.27666473388672, Test Loss: 0.2728966176509857, Test Accuracy: 90.13999938964844
Epoch 8, Loss: 0.16866986453533173, Accuracy: 93.78166961669922, Test Loss: 0.26838719844818115, Test Accuracy: 90.70999908447266
Epoch 9, Loss: 0.15428490936756134, Accuracy: 94.21499633789062, Test Loss: 0.25668397545814514, Test Accuracy: 91.3499984741211
Epoch 10, Loss: 0.14165101945400238, Accuracy: 94.82333374023438, Test Loss: 0.2612230181694031, Test Accuracy: 91.22999572753906


2024-08-24 13:15:49.520417: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
